In [355]:
import random
import datetime
import csv

def generate_user_spending(seed=42):
    random.seed(seed)
    start_date = datetime.date(2025, 2, 1)
    spending_data = []

    for day in range(1000):
        current_date = start_date + datetime.timedelta(days=day)
        weekday = current_date.weekday()

        if random.random() < 0.6:
            continue

        num_transactions = random.choices([0, 1, 2, 3, 4, 5], weights=[0.1, 0.4, 0.3, 0.1, 0.05, 0.05])[0]
        if num_transactions == 0:
            continue

        for _ in range(num_transactions):
            if weekday < 5:
                hour = random.choices([12, 18, 20, 9, 15], weights=[0.3, 0.3, 0.2, 0.1, 0.1])[0]
            else:
                hour = random.choices([11, 14, 19, 10, 16], weights=[0.3, 0.3, 0.2, 0.1, 0.1])[0]

            minute = random.randint(0, 59)
            second = random.randint(0, 59)
            transaction_time = datetime.time(hour, minute, second)

            category = random.choices(
                ["groceries", "dining", "coffee", "transport", "entertainment", "shopping", "other"],
                weights=[0.25, 0.20, 0.15, 0.10, 0.10, 0.10, 0.10]
            )[0]

            if category == "groceries":
                price = round(random.uniform(20, 150), 2)
            elif category == "dining":
                price = round(random.uniform(15, 80), 2)
            elif category == "coffee":
                price = round(random.uniform(15, 40), 2)
            elif category == "transport":
                price = round(random.uniform(20, 200), 2)
            elif category == "entertainment":
                price = round(random.uniform(10, 50), 2)
            elif category == "shopping":
                price = round(random.uniform(20, 200), 2)
            else:
                price = round(random.uniform(5, 50), 2)

            spending_data.append({
                "date": current_date.strftime("%Y-%m-%d"),
                "time": transaction_time.strftime("%H:%M:%S"),
                "price": price
            })

    return spending_data

february_data = generate_user_spending()

with open('spending.csv', 'w', newline='') as csvfile:
    fieldnames = ['date', 'time', 'price']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(february_data)

print("Data saved to spending.csv")


Data saved to spending.csv


In [90]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential
from keras.regularizers import l1_l2, l2
from keras.optimizers import Adam
from keras.initializers import HeNormal
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

import sklearn
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

In [356]:
df = pd.read_csv('giving2veda.csv')
df.head()

,Unnamed: 0,DATE,WITHDRAWAL (DR)
0,0,2022-11-01,0.0
1,1,2022-11-02,25.0
2,2,2022-11-05,0.0
3,3,2022-11-16,2.0
4,4,2022-11-16,19.0


In [357]:
df = df.drop(columns=['Unnamed: 0'])

In [358]:
df = df.rename(columns = {'DATE': 'date',
          'WITHDRAWAL (DR)': 'price'})
df.head()

,date,price
0,2022-11-01,0.0
1,2022-11-02,25.0
2,2022-11-05,0.0
3,2022-11-16,2.0
4,2022-11-16,19.0


In [359]:
df.shape

(532, 2)

In [360]:
df['date'] = pd.to_datetime(df['date'])
date_range = pd.date_range(start=df['date'].min(), end=df['date'].max(), freq='D')
full_df = pd.DataFrame(date_range, columns=['date'])
df = pd.merge(full_df, df, on='date', how='left')
df['price'] = df['price'].fillna(0)


In [361]:
df = df.groupby('date').agg({
    'price': 'sum'
}).rename(columns={'price': 'total_price'})
df.reset_index(inplace=True)

In [362]:
df.shape

(851, 2)

In [363]:
df1 = df.copy()
df1['date'] = pd.to_datetime(df1['date'])

In [364]:
def extract_date_features(df, date_col):
    df[date_col] = pd.to_datetime(df[date_col])
    df['month'] = df[date_col].dt.month
    df['day_of_week'] = df[date_col].dt.dayofweek
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    
    return df
df = extract_date_features(df, 'date')

In [366]:
df.shape

(851, 17)

In [367]:
df['total_price'].describe()

count     851.000000
mean      117.249941
std       510.111668
min         0.000000
25%         0.000000
50%         0.000000
75%        20.000000
max      5000.000000
Name: total_price, dtype: float64

In [368]:
df.dropna(inplace=True)

In [369]:
df.sort_values('date', inplace=True)
df.set_index('date', inplace=True)

In [370]:
df.shape

(830, 16)

In [371]:
x.shape

(834, 15)

In [372]:
x = df.drop(columns=['total_price'])
y = df['total_price']

In [373]:
scale_x = StandardScaler()
scale_y = StandardScaler()
x_scaled = scale_x.fit_transform(x)
y_scaled = scale_y.fit_transform(y.values.reshape(-1,1))

In [387]:
l = []

#ST

In [412]:
df = df1.copy()

In [413]:
def extract_date_features(df, date_col):
    df[date_col] = pd.to_datetime(df[date_col])
    df['month'] = df[date_col].dt.month
    df['day_of_week'] = df[date_col].dt.dayofweek
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    
    return df
df = extract_date_features(df, 'date')

In [414]:
for i in list(range(1, 8)) + [14, 21]:
    df[f'lag_{i}'] = df['total_price'].shift(i)

In [415]:
df.dropna(inplace=True)

In [416]:
df.sort_values('date', inplace=True)
df.set_index('date', inplace=True)

In [417]:
df.shape

(832, 16)

In [418]:
x.shape

(832, 15)

In [419]:
scale_x = StandardScaler()
scale_y = StandardScaler()
x_scaled = scale_x.fit_transform(x)
y_scaled = scale_y.fit_transform(y.reshape(-1,1))

In [420]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled,y_scaled, test_size=0.2, random_state=42)

In [421]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 0.01, 0.001],
    'kernel': ['linear'],
    'epsilon': [0.1, 0.05],
    'gamma': ['auto'],
    'degree': [2,3]
}


grid_search = SVR(C=0.01, kernel='linear')
grid_search.fit(x_train, y_train)

print("Best Parameters:", grid_search.best_params_)


Best Parameters: {'C': 0.1, 'degree': 2, 'epsilon': 0.1, 'gamma': 'auto', 'kernel': 'linear'}


D:\Anaconda3\envs\tf_env\lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [422]:
y_pred = grid_search.predict(x_test)
y_pred_original = scale_y.inverse_transform(y_pred.reshape(-1,1))
y_test_original = scale_y.inverse_transform(y_test.reshape(-1,1))

In [423]:
score = (mean_squared_error(y_test_original, y_pred_original))**1/2
score

104188.33067752657

In [424]:
score = (mean_squared_error(y_test, y_pred))**1/2
score

0.39261069066372073

In [425]:
score = mean_absolute_error(y_test_original, y_pred_original)
score

130.38935317730625

In [426]:
score = mean_absolute_error(y_test, y_pred)
score

0.2531125339355423

In [427]:
x.shape

(832, 15)

In [428]:
from datetime import datetime, timedelta
last_date = pd.to_datetime(df1.iloc[-1,0])
next_day = last_date + timedelta(days=1)
next_data = {
    'month': next_day.month,
    'day_of_week': next_day.weekday(),
    'day_sin': np.sin(2 * np.pi * next_day.day / 31),
    'day_cos': np.cos(2 * np.pi * next_day.day / 31),
    'month_sin': np.sin(2 * np.pi * next_day.month / 12),
    'month_cos': np.cos(2 * np.pi * next_day.month / 12),
}
for i in list(range(1, 8)) + [14, 21]:
    next_data[f'lag_{i}'] = df.iloc[-i, 0] 
next_data = pd.DataFrame([next_data])
next_data_values = scale_x.fit_transform(next_data)
predicted_value = grid_search.predict(next_data_values)
predicted_value = scale_y.inverse_transform(predicted_value.reshape((-1,1)))
predicted_value

array([[51.48874352]])

In [408]:
next_data

,month,day_of_week,day_sin,day_cos,month_sin,month_cos,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_14,lag_21
0,3,6,0.394356,0.918958,1.0,6.123234e-17,51.552091,0.0,0.0,25.0,0.0,95.0,0.0,87.0,0.0


In [407]:
df1.iloc[-1,0]

Timestamp('2025-03-01 00:00:00')

In [409]:
y = np.append(y, predicted_value)
x = pd.concat([x, next_data], ignore_index=True)
next_data['total_price'] = predicted_value[0]
next_data['date'] = next_day
df1 = pd.concat([df1, next_data], ignore_index=True)

In [410]:
l.append(predicted_value[0])

In [411]:
l

[array([51.52590215])]

In [297]:
x.iloc[-2]

month            2.000000
day_of_week      4.000000
day_sin         -0.433884
day_cos         -0.900969
month_sin        0.866025
month_cos        0.500000
lag_1            0.000000
lag_2           25.000000
lag_3            0.000000
lag_4           95.000000
lag_5            0.000000
lag_6            0.000000
lag_7           40.000000
lag_14         500.000000
lag_21           0.000000
Name: 829, dtype: float64

In [298]:
x.iloc[-1]

month          3.000000e+00
day_of_week    6.000000e+00
day_sin        3.943559e-01
day_cos        9.189578e-01
month_sin      1.000000e+00
month_cos      6.123234e-17
lag_1          0.000000e+00
lag_2          0.000000e+00
lag_3          2.500000e+01
lag_4          0.000000e+00
lag_5          9.500000e+01
lag_6          0.000000e+00
lag_7          0.000000e+00
lag_14         0.000000e+00
lag_21         0.000000e+00
Name: 830, dtype: float64

In [140]:
df.iloc[-3,0]

25.0

In [131]:
df.iloc[-1]

total_price      0.000000
month            2.000000
day_of_week      4.000000
day_sin         -0.433884
day_cos         -0.900969
month_sin        0.866025
month_cos        0.500000
lag_1            0.000000
lag_2           25.000000
lag_3            0.000000
lag_4           95.000000
lag_5            0.000000
lag_6            0.000000
lag_7           40.000000
lag_14         500.000000
lag_21           0.000000
Name: 2025-02-28 00:00:00, dtype: float64

In [257]:
x.shape

(39, 15)

In [259]:
scale_x = MinMaxScaler()
scale_y = MinMaxScaler()
x_scaled = scale_x.fit_transform(x)
y_scaled = scale_y.fit_transform(y.values.reshape(-1,1))

In [260]:
def create_sequences(X, y, lookback):
    Xs, ys = [], []
    for i in range(len(X) - lookback):
        Xs.append(X[i:(i + lookback)])
        ys.append(y[i + lookback])
    return np.array(Xs), np.array(ys)
lookback = 7
x_seq, y_seq = create_sequences(x_scaled, y_scaled, lookback)

In [261]:
x_seq.shape

(32, 7, 15)

In [262]:
x_train, x_test, y_train, y_test = train_test_split(x_seq, y_seq, test_size=0.2)

In [263]:
def create_model(input_shape):
    model = Sequential()
    model.add(LSTM(16, activation='relu', kernel_initializer=HeNormal(),
                   kernel_regularizer=l1_l2(l1=0.0001, l2=0.00005),
                   input_shape=input_shape, return_sequences=False))
    model.add(Dropout(0.3))
    model.add(Dense(8, activation='relu', kernel_initializer=HeNormal(),
                    kernel_regularizer=l1_l2(l1=0.0001, l2=0.00005)))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_losses = []

for fold, (train_index, val_index) in enumerate(kf.split(x_seq)):
    x_train, x_val = x_seq[train_index], x_seq[val_index]
    y_train, y_val = y_seq[train_index], y_seq[val_index]

    model = create_model((x_seq.shape[1], x_seq.shape[2]))

    early_stopping = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=6, min_lr=1e-5)

    history = model.fit(x_train, y_train, epochs=200, batch_size=8,
                        validation_data=(x_val, y_val),
                        callbacks=[early_stopping, reduce_lr], verbose=1)

    val_loss = model.evaluate(x_val, y_val, verbose=0)
    fold_losses.append(val_loss)
    print(f"Fold {fold + 1} Validation Loss: {val_loss}")

print(f"Average Validation Loss: {np.mean(fold_losses)}")

Epoch 1/200
4/4 [==============================] - 2s 146ms/step - loss: 1.1772 - val_loss: 0.4858 - lr: 0.0010
Epoch 2/200
4/4 [==============================] - 0s 14ms/step - loss: 0.7016 - val_loss: 0.3215 - lr: 0.0010
Epoch 3/200
4/4 [==============================] - 0s 16ms/step - loss: 0.6448 - val_loss: 0.2300 - lr: 0.0010
Epoch 4/200
4/4 [==============================] - 0s 16ms/step - loss: 0.2958 - val_loss: 0.1951 - lr: 0.0010
Epoch 5/200
4/4 [==============================] - 0s 16ms/step - loss: 0.3544 - val_loss: 0.1845 - lr: 0.0010
Epoch 6/200
4/4 [==============================] - 0s 15ms/step - loss: 0.2442 - val_loss: 0.1875 - lr: 0.0010
Epoch 7/200
4/4 [==============================] - 0s 15ms/step - loss: 0.2718 - val_loss: 0.2005 - lr: 0.0010
Epoch 8/200
4/4 [==============================] - 0s 15ms/step - loss: 0.3180 - val_loss: 0.2134 - lr: 0.0010
Epoch 9/200
4/4 [==============================] - 0s 19ms/step - loss: 0.2414 - val_loss: 0.2226 - lr: 0.0010


In [264]:
y_pred_scaled = model.predict(x_test)
y_pred = scale_y.inverse_transform(y_pred_scaled)
y_test_original = scale_y.inverse_transform(y_test)


1/1 [==============================] - 0s 217ms/step


In [265]:
print(y_train.min(), y_train.max())
print(y_test.min(), y_test.max())

0.0 0.9999999999999999
0.0 0.6292732855680655


In [266]:
mae = mean_absolute_error(y_test_original, y_pred)
rmse = np.sqrt(mean_squared_error(y_test_original, y_pred))
print(f'MAE: {mae:.2f}, RMSE: {rmse:.2f}')

MAE: 91.76, RMSE: 105.16


In [267]:
for i in range(y_test.shape[0]):
    print(y_pred[i], y_test_original[i])

[111.750435] [0.]
[92.40611] [245.92]
[127.444626] [182.22]
[123.826836] [166.85]
[118.476006] [0.]
[91.33401] [102.62]
[64.472946] [213.94]


In [530]:
from datetime import datetime, timedelta
next_day = datetime.strptime('28-02-2025', '%d-%m-%Y') + timedelta(days=1)
next_data = {
    'month': next_day.month,
    'day_of_week': next_day.weekday(),
    'day_sin': np.sin(2 * np.pi * next_day.day / 31),
    'day_cos': np.cos(2 * np.pi * next_day.day / 31),
    'month_sin': np.sin(2 * np.pi * next_day.month / 12),
    'month_cos': np.cos(2 * np.pi * next_day.month / 12),
}
if df.shape[0] > 60:
    for i in list(range(1, 8)) + [14, 21]:
        next_data[f'lag_{i}'] = x_scaled[-i, 0]
else:
    for i in range(1, 8):
        next_data[f'lag_{i}'] = x_scaled[-i, 0]
next_day_input = pd.DataFrame([next_data])
next_day_input = scale_x.transform(next_day_input)
x_input = next_day_input.reshape(1, 1, -1)
predicted_value = model.predict(x_input)
original_predicted_value = scale_y.inverse_transform(predicted_value)
print(f"Predicted Value for {next_day.strftime('%d-%m-%Y')}: {original_predicted_value[0][0]}")

1/1 [==============================] - 0s 372ms/step
Predicted Value for 01-03-2025: 22.104232788085938


In [531]:
last_week_data = y[-7:]
print("Last Week's Spending Data:", last_week_data)

Last Week's Spending Data: date
2025-02-22     54.80
2025-02-23     36.89
2025-02-24    134.71
2025-02-25    310.15
2025-02-26     98.97
2025-02-27    290.49
2025-02-28     65.20
Name: total_price, dtype: float64


In [532]:
df[df['day_of_week'] == 5]

,total_price,month,day_of_week,day_sin,day_cos,month_sin,month_cos,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7
date,,,,,,,,,,,,,,
2025-02-01,45.62,2,5,-0.974928,-0.222521,0.866025,0.5,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2025-02-08,252.06,2,5,-0.974928,-0.222521,0.866025,0.5,381.57,199.31,327.42,184.70,86.60,313.99,45.62
2025-02-15,0.00,2,5,-0.974928,-0.222521,0.866025,0.5,131.32,0.00,156.48,279.49,99.73,85.45,252.06
2025-02-22,54.80,2,5,-0.974928,-0.222521,0.866025,0.5,133.08,76.83,219.04,333.34,0.00,239.09,0.00
